In [1]:
from sklearn import datasets
from nltk.corpus import stopwords
import random
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize import TreebankWordTokenizer
import pickle

In [2]:
import nltk
import pandas as pd
import numpy as np
import math
from nltk import pos_tag
import matplotlib.pyplot as plt

In [3]:
train=pd.read_csv('train_twitter.csv')
test=pd.read_csv('test_twitter.csv')

In [4]:
train.columns

Index(['tweet_id', 'airline_sentiment', 'airline', 'airline_sentiment_gold',
       'name', 'negativereason_gold', 'retweet_count', 'text', 'tweet_coord',
       'tweet_created', 'tweet_location', 'user_timezone'],
      dtype='object')

In [5]:
y_train=train['airline_sentiment']
x_train=train.loc[:,['text','airline_sentiment_gold']]

In [6]:
x_test=test.loc[:,['text','airline_sentiment_gold']]

In [7]:
x_train.iloc[:,1]=(x_train.iloc[:,1]).fillna(0)
x_test.iloc[:,1]=(x_test.iloc[:,1]).fillna(0)

In [8]:
#x_train.iloc[:,0]

In [9]:
stopWords = stopwords.words('english')
punct=list(string.punctuation)
lemm = WordNetLemmatizer()
stopWords=stopWords+punct

In [10]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return 'a'
    elif treebank_tag.startswith('V'):
        return 'v'
    elif treebank_tag.startswith('N'):
        return 'n'
    elif treebank_tag.startswith('R'):
        return 'r'
    else:
        return ''

In [11]:
def lemmatization(pos):
        l1=[]
        for word in pos:
            #print(type(word))
            word=list(word)
            a=get_wordnet_pos(word[1])
            try:
                new_word=lemm.lemmatize(word[0],pos=a)
                l1.append(new_word)
            except:
                l1.append(word[0])
        return l1   

In [12]:
def all_words(doc):
    l1=(word_tokenize(doc))
    final_doc=[]
    for l in l1:
        l=''.join([i for i in l if not i.isdigit()])
        if l.startswith('@') :
            l1.remove(l)
        else:
            for c in string.punctuation:
           
                l=l.replace(c,"")
            if len(l)>3:
                if l.lower() not in stopWords:
                     if not l.endswith("edu"):
                               final_doc.append(l.lower())
    return(final_doc)
   

In [13]:
data=[]
for i in range(x_train.shape[0]):
    sentence=all_words(x_train.iloc[i,0])
    data.append((" ".join(sentence)))
    

In [14]:
test_data=[]
for i in range(x_test.shape[0]):
    sentence=all_words(x_test.iloc[i,0])
    test_data.append((" ".join(sentence)))
    

In [15]:
#(len(data)),y_train.shape

In [16]:
document=[]
for i in range(y_train.shape[0]):
    document.append((data[i],y_train[i]))

In [17]:
random.shuffle(document)
train=document[:8784]
val=document[8784:]

In [18]:
xtrain=[]
ytrain=[]
for i in (train):
    xtrain.append(i[0])
    ytrain.append(i[1])
    

In [19]:
xtest=[]
ytest=[]
for j in (val):
    xtest.append(j[0])
    ytest.append(j[1])
    

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(max_features=3200)
data1 = vec.fit_transform(xtrain).toarray()
print(data1)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [21]:
test_a=vec.transform(xtest).toarray()

In [22]:
testing_data=vec.transform(test_data).toarray()

In [23]:
xtrain=pd.DataFrame(data1)
xtest=pd.DataFrame(test_a)

In [24]:
ytrain=pd.DataFrame(ytrain)
ytest=pd.DataFrame(ytest)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV

C:\Users\Varsha\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\Varsha\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Varsha\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning

In [61]:
rf=RandomForestClassifier()
rf.fit(xtrain,ytrain)
rf.score(xtest,ytest)

C:\Users\Varsha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.7390710382513661

In [26]:
l=LogisticRegression(fit_intercept=False)
l.fit(xtrain,ytrain)
l.score(xtest,ytest)

C:\Users\Varsha\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.75591985428051

In [63]:
d=DecisionTreeClassifier()
d.fit(xtrain,ytrain)
d.score(xtest,ytest)

0.697632058287796

In [64]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [65]:
g=BernoulliNB()
g.fit(xtrain,ytrain)
g.score(xtest,ytest)

C:\Users\Varsha\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7764116575591985

In [66]:
predict=l.predict(testing_data)

In [67]:
predicted=pd.DataFrame(predict)

In [68]:
predicted.to_csv('twitterc.csv',index=False,header=None)